### Applied Data Science Capstone Project

In [1]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


**Week 1 - Introduction to Capstone Project**<br>
    Introduction to Capstone Project<br>
    Location Data Providers<br>
    Signing-up for a Watson Studio Account<br>
    Peer-review Assignment: Capstone Project Notebook<br>
    
**Week 2 - Foursquare API**<br>
    Introduction to Foursquare<br>
    Getting Foursquare API Credentials<br>
    Using Foursquare API<br>
    Lab: Foursquare API<br>
    Quiz: Foursquare API<br>
    
**Week 3 - Neighborhood Segmentation and Clustering**<br>
    Clustering<br>
    Lab: Clustering<br>
    Lab: Segmenting and Clustering Neighborhoods in New York City<br>
    Peer-review Assignment: Segmenting and Clustering Neighborhoods in Toronto<br>
    
**Week 4 - Capstone Project**<br>

**Week 5 - Capstone Project (Cont'd)**<br>

In [2]:
import pandas as pd
import numpy as np